In [9]:
# 주요 라이브러리 불러오기
import requests  # 웹에서 데이터 가져올 때 사용
import openai  # OpenAI API를 사용하기 위한 라이브러리
from openai import OpenAI  # 클라이언트 객체 생성용
from typing import Dict, List, Tuple  # 타입 힌트 지원
import time  # 시간 측정용
import re  # 정규표현식 (현재 코드에서는 사용되지 않음)
import pandas as pd  # 데이터프레임 작업용
import tiktoken  # 토큰 수 계산 (GPT의 입력 길이 측정용)
import random
from dataclasses import dataclass
from dotenv import load_dotenv
import os
from pathlib import Path

# OpenAI API 설정
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [10]:
# 테스트 결과를 저장할 데이터 클래스 정의
dataclass
class TestResult:
    model: str                      # 사용한 모델 이름 (예: gpt-4o)
    prompt_method: str             # 사용한 프롬프트 기법
    correct_answers: int          # 정답 개수
    total_questions: int          # 전체 문제 수
    score: float                  # 정답률 (%)
    total_score: float            # 총 점수 (문제별 점수 합산)
    execution_time: float         # 실행 시간 (초)
    detailed_results: pd.DataFrame  # 개별 문제 결과 상세 정보


In [11]:

# prompts.py로 뺄 수도 있음
PROMPTS = {
    "zero_shot": """다음 수능 국어 문제를 풀어주세요.
    
    지문:
    {paragraph}
    
    문제: {question}
    
    {question_plus}
    
    선택지:
    {choices1}
    {choices2}
    {choices3}
    {choices4}
    {choices5}
    
    위 문제를 신중히 분석하고 정답을 선택해주세요.
    정답: """,
    
    "emotional_appeal": """🎯 인생에서 최고로 중요한 수능 국어 문제입니다.
    당신이 소중하게 생각하는 할머니께서 당신이 좋은 성적을 얻기만을 기다리고 계십니다. 당신의 실력을 보여주세요.
    이 문제는 많은 수험생들이 어려워하는 유형입니다. 하지만 당신은 분명히 풀 수 있습니다! 
    차근차근 읽고, 단계별로 생각해서 최선의 답을 찾아주세요.
    자신감을 가지고 열정적으로 풀어보세요.
    
    지문:
    {paragraph}
    
    ❓ 문제: {question}
    
    {question_plus}
    
    🔍 선택지를 신중하게 검토해보세요:
    {choices1}
    {choices2}
    {choices3}
    {choices4}
    {choices5}
    
    🧠 깊이 생각해보고, 지문의 내용을 정확히 파악한 후 답을 선택하세요.
    당신은 반드시 정답을 찾을 수 있습니다.
    
    이 문제를 틀리면 정말 좋은 기회를 놓치게 될 것입니다. 신중하게, 그리고 확신을 가지고 답해주세요.
    
    💯 최종 정답: """,
    
    "expert_role": """당신은 월 9900억원을 받고 일하는 한국교육과정평가원의 수능 국어 출제위원입니다.
    - 30년간 수능 국어 출제 경험을 보유한 최고 전문가
    - 국어교육학 박사, 문학 박사 학위 소지
    - 매년 수능 출제와 검토에 참여하는 국어 교육 권위자
    
    출제위원으로서 이 문제를 전문적으로 분석하고 해결해주세요.
    
    【지문】
    {paragraph}
    
    【문제】
    {question}
    
    【추가 자료】
    {question_plus}
    
    【선택지】
    {choices1}
    {choices2}
    {choices3}
    {choices4}
    {choices5}
    
    【전문가 분석 과정】
    1. 출제 의도 파악: 이 문제가 평가하고자 하는 능력은?
    2. 지문 핵심 내용: 문제 해결의 열쇠가 되는 지문의 핵심은?
    3. 선택지 분석: 각 선택지의 옳고 그름을 판단하는 근거는?
    4. 정답 근거: 명확한 텍스트 근거를 바탕으로 한 정답은?
    
    【출제위원 최종 판단】
    정답: """
    }



In [12]:
class KoreanSuneungSolver:
    def __init__(self):
        self.data = None
        self.results = {}
        self.client = OpenAI()
        self.tokenizer = tiktoken.encoding_for_model("gpt-4o")
        
    def load_data(self):
        """GitHub에서 2023년 수능 국어 데이터 로드"""
        github_url = "https://raw.githubusercontent.com/NomaDamas/KICE_slayer_AI_Korean/master/data/2023_11_KICE.json"
        try:
            response = requests.get(github_url, timeout=30)
            response.raise_for_status()
            raw_data = response.json()
            
            # 코드를 반영한 데이터 구조 변환
            total_result = []
            for data in raw_data:
                paragraph_id = data['id']
                paragraph = data['paragraph']
                data_type = data['type']
                problems_lst = data['problems']
                
                for problem in problems_lst:
                    question = problem['question']
                    # 선택지를 "1번 내용" 형태로 변환
                    choices = [f"{i+1}번 {choice}" for i, choice in enumerate(problem['choices'])]
                    answer = problem['answer']
                    score = problem['score']
                    question_plus = problem.get('question_plus', '')
                    
                    problem_dict = {
                        'paragraph_id': paragraph_id,
                        'type': data_type,
                        'paragraph': paragraph,
                        'question': question,
                        'question_plus': question_plus,
                        'choices': choices,
                        'answer': answer,
                        'score': score,
                    }
                    total_result.append(problem_dict)
            
            self.data = total_result
            print(f"데이터 로드 완료: {len(self.data)} 문제")
            return True
        except Exception as e:
            print(f"데이터 로드 실패: {e}")
            return False
    
    def normalize_answer(self, text: str) -> int:
        """답안 정규화"""
        if hasattr(text, 'content'):
            result = text.content
        else:
            result = str(text)
        
        result = result.split("정답:")[-1]
        for num in range(1, 6):
            if str(num) in result:
                return num
        
        return random.randint(1, 5)
    
    # 모두 고르시오 문제에 해당하는 예외처리 혹은 수정 필요
    
    
    def prediction(self, prompt: str, model: str = "gpt-4o"):
        """OpenAI API 호출"""
        try:
            completion = self.client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}]
            )
            return completion.choices[0].message
        except Exception as e:
            print(f"API 호출 실패: {e}")
            return None
    
    def build_prompts(self, prompt_name: str) -> List[str]:
        """특정 프롬프트 타입으로 모든 문제에 대한 프롬프트 생성"""
        if prompt_name not in PROMPTS:
            raise ValueError(f"지원하지 않는 프롬프트: {prompt_name}")
        
        prompt_template = PROMPTS[prompt_name]
        prompt_lst = []
        
        for data in self.data:
            # question_plus가 비어있을 경우 처리
            question_plus_text = f"추가 정보: {data['question_plus']}" if data['question_plus'] else ""
            
            prompt = prompt_template.format(
                paragraph=data['paragraph'],
                question=data['question'],
                question_plus=question_plus_text,
                choices1=data['choices'][0],
                choices2=data['choices'][1],
                choices3=data['choices'][2],
                choices4=data['choices'][3],
                choices5=data['choices'][4],
            )
            prompt_lst.append(prompt)
        
        return prompt_lst
    
    def run_test(self, prompt_name: str, model: str = "gpt-4o", max_questions: int = None) -> TestResult:
        """프롬프트로 전체 테스트 실행"""
        if self.data is None:
            raise ValueError("데이터를 먼저 로드해주세요.")
        
        test_data = self.data[:max_questions] if max_questions else self.data
        prompt_lst = self.build_prompts(prompt_name)[:len(test_data)]
        
        print(f"\n테스트 시작: {model} + {prompt_name}")
        
        start_time = time.time()
        pred_lst = []
        answer_lst = []
        score_lst = []
        input_token_lst = []
        output_token_lst = []
        response_lst = []
        
        for i, (prompt, problem) in enumerate(zip(prompt_lst, test_data)):
            try:
                # 진행률 표시
                self.print_progress(i + 1, len(test_data), "문제 풀이 중")
                
                # 예측
                result = self.prediction(prompt, model)
                if result is None:
                    pred = random.randint(1, 5)
                    response_text = "API 호출 실패"
                else:
                    pred = self.normalize_answer(result)
                    response_text = result.content
                
                pred_lst.append(pred)
                answer_lst.append(problem['answer'])
                score_lst.append(problem['score'])
                input_token_lst.append(len(self.tokenizer.encode(prompt)))
                output_token_lst.append(len(self.tokenizer.encode(response_text)) if result else 0)
                response_lst.append(response_text)
                
                # API 제한 고려하여 잠시 대기
                time.sleep(0.5)
                
            except Exception as e:
                print(f"\n문제 {i+1} 처리 중 오류: {e}")
                pred_lst.append(random.randint(1, 5))
                answer_lst.append(problem['answer'])
                score_lst.append(problem['score'])
                input_token_lst.append(0)
                output_token_lst.append(0)
                response_lst.append(f"오류: {str(e)}")
        
        end_time = time.time()
        execution_time = end_time - start_time
        
        # DataFrame 생성 (멘토님 코드 구조 반영)
        df = pd.DataFrame({
            'pred': pred_lst,
            'answer': answer_lst,
            'score': score_lst,
            'input_tokens': input_token_lst,
            'output_tokens': output_token_lst,
            'response': response_lst,
            'time': execution_time,
        })
        
        # 정답 여부 계산
        df['result'] = df['pred'].apply(lambda x: int(x)) == df['answer'].apply(lambda x: int(x))
        
        # 결과 통계
        correct_answers = df['result'].sum()
        total_questions = len(df)
        total_score = (df['result'] * df['score']).sum()
        percentage = (correct_answers / total_questions) * 100
        
        # CSV 저장
        os.makedirs("data/results", exist_ok=True)
        csv_path = f"data/results/{prompt_name}_{model}.csv"
        df.to_csv(csv_path, index=False, encoding='utf-8')
        
        result = TestResult(
            model=model,
            prompt_method=prompt_name,
            correct_answers=correct_answers,
            total_questions=total_questions,
            score=percentage,
            total_score=total_score,
            execution_time=execution_time,
            detailed_results=df
        )
        
        self.results[f"{prompt_name}_{model}"] = result
        
        print(f"✅ 완료: {prompt_name} + {model}")
        print(f"   점수: {total_score}점, 정답률: {percentage:.1f}% ({correct_answers}/{total_questions})")
        print(f"   소요시간: {execution_time:.3f}초")
        
        return result
    
    def print_progress(self, current: int, total: int, desc: str = "진행중"):
        """진행률 출력 함수"""
        percentage = (current / total) * 100
        bar_length = 30
        filled_length = int(bar_length * current // total)
        bar = '█' * filled_length + '-' * (bar_length - filled_length)
        print(f'\r{desc}: |{bar}| {current}/{total} ({percentage:.1f}%)', end='', flush=True)
        if current == total:
            print()
    
    def run_comprehensive_test(self, models: List[str] = None, max_questions: int = None):
        """종합 테스트 실행"""
        if models is None:
            models = ["gpt-4o-mini", "gpt-4o"]
        
        prompt_methods = ["zero_shot", "emotional_appeal", "expert_role"]
        
        print("=" * 60)
        print("🎯 2023 수능 국어 AI 풀이 프로그램")
        print("=" * 60)
        print(f"📊 테스트할 문제 수: {max_questions if max_questions else len(self.data)}")
        print(f"🤖 모델: {', '.join(models)}")
        print(f"💡 프롬프트 기법: {', '.join(prompt_methods)}")
        print("=" * 60)
        
        for model in models:
            for prompt_method in prompt_methods:
                try:
                    self.run_test(prompt_method, model, max_questions)
                    print("-" * 40)
                except Exception as e:
                    print(f"❌ 테스트 실패 - {model} + {prompt_method}: {e}")
    
    def analyze_results(self):
        """결과 분석 및 출력"""
        if not self.results:
            print("분석할 결과가 없습니다.")
            return
        
        print("\n" + "="*60)
        print("📊 최종 결과 분석")
        print("="*60)
        
        # 각 프롬프트에 대한 결과 요약
        print("\n📋 프롬프트별 결과 요약:")
        for name, result in self.results.items():
            print(f"  {name}: 점수 {result.total_score}, 정답률 {result.score:.1f}%, 시간 {result.execution_time:.3f}초")
        
        # 최고 성능
        print(f"\n🏆 최고 성능: {best_result.model} + {best_result.prompt_method}")
        print(f"   점수: {best_result.total_score}점 ({best_result.correct_answers}/{best_result.total_questions})")
        
        # 상세 분석 (멘토님 코드의 정답/오답 예시 방식)
        self.show_sample_responses()
    
    def show_sample_responses(self):
        """각 프롬프트 타입에 대한 정답과 오답의 예시"""
        print(f"\n🔍 프롬프트별 정답/오답 예시 분석:")
        
        for i, (name, result) in enumerate(self.results.items()):
            df = result.detailed_results
            
            # 정답 예시
            correct_samples = df[df['result'] == True]
            incorrect_samples = df[df['result'] == False]
            
            if len(correct_samples) > 0 and len(incorrect_samples) > 0:
                print(f"\n{'#'*5}")
                print(f"{i+1}번째: {name} 프롬프트 분석\n")
                
                # 정답 예시
                true_sample = correct_samples.iloc[0]
                print(f"✅ 정답 예시:")
                print(f"  예측: {true_sample['pred']}")
                print(f"  정답: {true_sample['answer']}")
                print(f"  응답: {true_sample['response'][:200]}...\n")
                
                # 오답 예시
                false_sample = incorrect_samples.iloc[0]
                print(f"❌ 오답 예시:")
                print(f"  예측: {false_sample['pred']}")
                print(f"  정답: {false_sample['answer']}")
                print(f"  응답: {false_sample['response'][:200]}...\n")
    
    def export_detailed_results(self):
        """상세 결과를 JSON과 CSV로 내보내기"""
        if not self.results:
            print("내보낼 결과가 없습니다.")
            return
        
        # JSON 형태로 요약 결과 저장
        summary_data = []
        for name, result in self.results.items():
            summary_data.append({
                'model': convert_np(result.model),
                'prompt_method': convert_np(result.prompt_method),
                'total_score': convert_np(result.total_score),
                'score_percentage': convert_np(result.score),
                'correct_answers': convert_np(result.correct_answers),
                'total_questions': convert_np(result.total_questions),
                'execution_time': convert_np(result.execution_time)
            })

        
        os.makedirs("data/results", exist_ok=True)
        with open("data/results/summary_results.json", 'w', encoding='utf-8') as f:
            json.dump(summary_data, f, ensure_ascii=False, indent=2)
        
        print(f"\n💾 결과 저장 완료:")
        print(f"   - 요약: data/results/summary_results.json")
        print(f"   - 상세: data/results/ 폴더의 각 CSV 파일들")

def main():
    """메인 실행 함수"""
    solver = KoreanSuneungSolver()
    
    # API 키 확인
    if not os.getenv("OPENAI_API_KEY"):
        print("❌ OpenAI API 키가 설정되지 않았습니다.")
        print("OPENAI_API_KEY 환경변수를 설정하거나 .env 파일을 확인해주세요.")
        return
    
    # 1. 데이터 로드
    print("📥 데이터 로딩 중...")
    if not solver.load_data():
        print("❌ 데이터 로드에 실패했습니다. GitHub 연결을 확인해주세요.")
        return
    
    # 2. 테스트 문제 수 설정
    total_problems = len(solver.data)
    print(f"전체 문제 수: {total_problems}")
    print("테스트할 문제 수를 입력하세요 (엔터 시 전체): ", end="")
    try:
        user_input = input().strip()
        if user_input:
            test_count = int(user_input)
            test_count = min(test_count, total_problems)
        else:
            test_count = None  # 전체 테스트
    except:
        test_count = 20  # 기본값
        print(f"기본값 {test_count}개 문제로 설정됩니다.")
    
    # 3. 종합 테스트 실행
    solver.run_comprehensive_test(max_questions=test_count)
    
    # 4. 결과 분석
    solver.analyze_results()
    
    # 5. 결과 내보내기
    solver.export_detailed_results()
    
    print(f"\n🎉 테스트 완료!")

if __name__ == "__main__":
    main()

📥 데이터 로딩 중...
데이터 로드 완료: 45 문제
전체 문제 수: 45
테스트할 문제 수를 입력하세요 (엔터 시 전체): 

🎯 2023 수능 국어 AI 풀이 시스템 (개선된 버전)
📊 테스트할 문제 수: 45
🤖 모델: gpt-4o-mini, gpt-4o
💡 프롬프트 기법: zero_shot, emotional_appeal, expert_role

테스트 시작: gpt-4o-mini + zero_shot
문제 풀이 중: |██████████████████████████████| 45/45 (100.0%)
✅ 완료: zero_shot + gpt-4o-mini
   점수: 49점, 정답률: 48.9% (22/45)
   소요시간: 455.430초
----------------------------------------

테스트 시작: gpt-4o-mini + emotional_appeal
문제 풀이 중: |██████████████████████████████| 45/45 (100.0%)
✅ 완료: emotional_appeal + gpt-4o-mini
   점수: 46점, 정답률: 46.7% (21/45)
   소요시간: 532.678초
----------------------------------------

테스트 시작: gpt-4o-mini + expert_role
문제 풀이 중: |██████████████████████████████| 45/45 (100.0%)
✅ 완료: expert_role + gpt-4o-mini
   점수: 54점, 정답률: 53.3% (24/45)
   소요시간: 666.171초
----------------------------------------

테스트 시작: gpt-4o + zero_shot
문제 풀이 중: |██████████████████████████████| 45/45 (100.0%)
✅ 완료: zero_shot + gpt-4o
   점수: 67점, 정답률: 68.9% (31/45)
   소요시간: 258.456초
----------------------------------------

테스트 시작: gpt-4o + emotiona

NameError: name 'convert_np' is not defined